In [ ]:
%matplotlib inline


# Neuropop Example


A demonstration of Neuropop using simulated data




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spykes.ml.neuropop import NeuroPop
from spykes.utils import train_test_split

Create a NeuroPop object
-----------------------------



In [ ]:
n_neurons = 10
pop = NeuroPop(n_neurons=n_neurons, tunemodel='glm')

Simulate a population of neurons
-----------------------------



In [ ]:
n_samples = 1000
x, Y, mu, k0, k, g, b = pop.simulate(pop.tunemodel, n_samples=n_samples,
                                     winsize=400.0)

Split into training and testing sets
-----------------------------



In [ ]:
np.random.seed(42)
(Y_train, Y_test), (x_train, x_test) = train_test_split(Y, x, percent=0.5)

Fit the tuning curves with gradient descent
-----------------------------



In [ ]:
pop.fit(x_train, Y_train)

Predict the population activity with the fit tuning curves
-----------------------------



In [ ]:
Yhat_test = pop.predict(x_test)

Score the prediction
-----------------------------



In [ ]:
Ynull = np.mean(Y_train, axis=0)
pseudo_R2 = pop.score(Y_test, Yhat_test, Ynull, method='pseudo_R2')
print(pseudo_R2)

Plot the simulated and fit tuning curves
-----------------------------



In [ ]:
plt.figure(figsize=[15, 15])

for neuron in range(pop.n_neurons):
    plt.subplot(4, 3, neuron + 1)
    pop.display(x_test, Y_test[:, neuron], neuron=neuron,
                ylim=[0.8 * np.min(Y_test[:, neuron]), 1.2 *
                np.max(Y_test[:, neuron])])

plt.show()

Decode feature from the population activity
-----------------------------



In [ ]:
xhat_test = pop.decode(Y_test)

Visualize ground truth vs. decoded estimates
-----------------------------



In [ ]:
plt.figure(figsize=[6, 5])

plt.plot(x_test, xhat_test, 'k.', alpha=0.5)
plt.xlim([-1.2 * np.pi, 1.2 * np.pi])
plt.ylim([-1.2 * np.pi, 1.2 * np.pi])
plt.xlabel('Ground truth [radians]')
plt.ylabel('Decoded [radians]')
plt.tick_params(axis='y', right='off')
plt.tick_params(axis='x', top='off')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.figure(figsize=[15, 5])
jitter = 0.2 * np.random.rand(x_test.shape[0])
plt.subplot(132, polar=True)
plt.plot(x_test, np.ones(x_test.shape[0]) + jitter, 'ko', alpha=0.5)
plt.title('Ground truth')

plt.subplot(133, polar=True)
plt.plot(xhat_test, np.ones(xhat_test.shape[0]) + jitter, 'co', alpha=0.5)
plt.title('Decoded')
plt.show()

Score decoding performance
-----------------------------



In [ ]:
# Circular correlation
circ_corr = pop.score(x_test, xhat_test, method='circ_corr')
print('Circular correlation: %f' % (circ_corr))

In [ ]:
# Cosine distance
cosine_dist = pop.score(x_test, xhat_test, method='cosine_dist')
print('Cosine distance: %f' % (cosine_dist))